In [ ]:
# * Import dataset
#     * What datasets???
# * Data Cleaning
# * Open AI SDK
# * Kick off a fine tune
# * Can I share a pre-built fine tune that others can use?
# * Test on some live examples
# * Demonstrate adding more training data to an existing fine tune

# Do a section on Prompt Engineering too
# Compare how prompt engineering vs fine tuning compares for out-of-sample data using sms+youtube data

In [22]:
import pandas as pd
import openai
import os
from dotenv import load_dotenv
load_dotenv()
from src.util import distributionPreservingDownsample

In [15]:
apikey = os.environ['OPENAI_API_KEY']
openai.api_key = apikey

In [25]:
sms_spam_all = pd.read_csv('../data/kaggle_sms_spam.csv', encoding='latin-1')[['label', 'prompt']]
sms_spam_all['spam_flag'] = sms_spam_all['label'].apply(lambda x: True if x == 'spam' else False)
sms_spam = sms_spam_all.drop_duplicates(subset=['prompt'])
print("Loaded sms data file with {} rows, kept {}".format(len(sms_spam_all), len(sms_spam)))
sms_spam.head()


Loaded sms data file with 5572 rows, kept 5169


,label,prompt,spam_flag
0,ham,"Go until jurong point, crazy.. Available only ...",False
1,ham,Ok lar... Joking wif u oni...,False
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,True
3,ham,U dun say so early hor... U c already then say...,False
4,ham,"Nah I don't think he goes to usf, he lives aro...",False


# Downsample to smaller dataset

When first training the model, it can be helpful to work with a smaller dataset.  This distributionPreservingDownsample will select a random subset of the dataframe, with the same ratio of spam/ham in the dataset.

In [26]:
downsampled = distributionPreservingDownsample(sms_spam, 'spam_flag', 500)
print("Downsampled {} -> {}".format(len(sms_spam),len(downsampled)))
print("Ratio of ham/spam in original dataset: {} / {}".format(len(sms_spam[sms_spam['spam_flag'] == True]) / len(sms_spam),len(sms_spam[sms_spam['spam_flag'] == False]) / len(sms_spam)))
print("Ratio of ham/spam in downsampled dataset: {} / {}".format(len(downsampled[downsampled['spam_flag'] == True]) / len(downsampled),len(downsampled[downsampled['spam_flag'] == False]) / len(downsampled)))


Downsampled 5169 -> 500
Ratio of ham/spam in original dataset: 0.12633004449603405 / 0.873669955503966
Ratio of ham/spam in downsampled dataset: 0.126 / 0.874
